# What is about ?


Look at modularity of KNN graphs for single cell data with Hubness reduction and without , also testing other transforms.

V25 - same as 24, but for mouse 

V24 - human - analysis of pca-reduction to many different values 

V23 - same as 22 , but for mouse 

V22 same, but  transforms : None and log  (again as before). Also new aggregated statistics output added 

V21 human - all with more than 200 cells, fixed: transform = log 

V20 - skip 

V19 - human again , datasets with 200 - 1000 cells

V18 same as v17 but datasets 200-500 cells 

V17 same as V16 but datasets with 500-1000 cells 

V16 same as V15 but for MOUSE (not human) data - 38 datasets 

V15 same as V14 but PCA 50 and 200 (not only 50 as in V14). 

V14 analyse several datasets  from collection:  ARCHS4/human/ cell number: more than 1000 - 27 datasets 
(see info in https://www.kaggle.com/alexandervc/multiple-single-cell-rna-expressions-archs4 )

V13 cancelled - more than 8 hours  dataset = 'GSE95753_v5_scrna_10x_6000_19585_Mouse_Brain_DentateGyrus_invivo_fromCytotrace'

V12 dataset = 'Torus1000x10000'

V11 dataset = 'Sphere1000x10000'

V10 dataset = 'Uniform1000x10000'

V8,9 Gaussian 1000x10_000

V7 - dataset = 'Dexter300from_scikit_hubness'

V6 - dataset = 'openml1146OVA_Prostate'

V5 - dataset = 'GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace'

V4 - dataset = 'GSE90047_Mouse_Liver_Hepatoblast_invivo'

V3 - dataset = 'single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi'

V2: extend by transforms by PCA, extend statistics

V1: Data - 'human_liver_ARCHS4' Conclusion - best modularity - mutual_proximity minkowski log 0.908534

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import h5py

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import time

_t00 = time.time() # script start time 

# Open file

In [ ]:
#filename = '/kaggle/input/multiple-single-cell-rna-expressions-archs4/human_matrix.h5'
filename = '/kaggle/input/multiple-single-cell-rna-expressions-archs4/mouse_matrix.h5'

f = h5py.File(filename,'r')#, mode)
for key in f.keys():
    print(key) #Names of the groups in HDF5 file.

# Access full expression data  

In [ ]:
key = 'data'
key2 = 'expression'
X_full = f[key][key2]
print(X_full.shape)
X_full

In [ ]:
X_full[:5,:8]

# Look at different GSE present in data


In [ ]:
# Extract GSE and look how many unique we have:
key = 'meta'
group = f[key]
d = group['Sample_series_id']
print(  len(np.unique(d) ) )
series_GSE_for_each_row = pd.Series(d)
v = series_GSE_for_each_row.value_counts()
print( (v > 2000).sum(), (v > 1000).sum(),  (v > 500).sum(),(v > 100).sum(), (v<=10).sum(), (v==1).sum(), )
v

In [ ]:
# Some statistics on number of cells corresponding to different GSE:
series_GSE_and_cell_count = series_GSE_for_each_row.value_counts()
series_GSE_and_cell_count.describe()

# Generate statistics for datasets with more than 1000 cells

In [ ]:
!pip install scikit-hubness


In [ ]:
from skhubness.neighbors import NearestNeighbors
from sklearn.neighbors import NearestNeighbors as NearestNeighbors_sklearn
from sklearn.decomposition import PCA

import igraph

In [ ]:
verbose = 0 

#mm = (1500 < series_GSE_and_cell_count ) &  ( series_GSE_and_cell_count < 2000 )
mm = (301 < series_GSE_and_cell_count ) &  ( series_GSE_and_cell_count <= 500000 )

n_neighbors = 10

t00 = time.time()

print( mm.sum() )
#print( series_GSE_and_cell_count[mm] )

key = 'meta'
key2 = 'gene_name'
gene_names = f[key][key2]

list_results = []
df_stat = pd.DataFrame()
ix4df_stat = 0
for GSE in series_GSE_and_cell_count[mm].index:
    print()
    print(GSE)
    t0 = time.time()
    mask = GSE == series_GSE_for_each_row
    mask = mask.values
    X = X_full[mask,:]
    print(X.shape, 'count totally zero genes',  (X.sum(axis = 0) == 0 ).sum() ) 
    m = X.sum(axis = 0) != 0 
    X = X[:,m]

    
    for dim_reduction in [None, 50, 100, 150,200, 250, 300]: #None, , 200 10, 50, 100, 500]:
        for transform_type in [ 'log']: # '',
            for metric in ['cosine'  ]: # 'minkowski', ,   'minkowski'
                for mode in ['Standard KNN graph' ,  'mutual_proximity']:# ,   ,  "local_scaling" "local_scaling", "dis_sim_local" ] : # 'KNN graph with hubness = mutual_proximity' ]: # range(2):
                    if verbose >= 10:
                        print()
                    str_info = str(GSE) + ' ' +mode + ' ' +  metric + ' ' + transform_type 
                    if dim_reduction is not None: str_info += 'PCA'+str(dim_reduction)
                    if str_info[-1]==' ': str_info = str_info[:-1] 
                    
                    df_stat.loc[ ix4df_stat, 'GSE'] = GSE

                    t0 = time.time()

                    X2 = X.copy()

                    if transform_type == 'log':
                        X2 = np.log(1+np.abs(X2) ) 

                    if dim_reduction is not None:
                        pca = PCA(n_components=dim_reduction)
                        pca.fit(X2)
                        X2 = pca.transform(X2)

                    if mode == 'Standard KNN graph':
                        nbrs = NearestNeighbors_sklearn(n_neighbors=n_neighbors, metric = metric).fit(X2) #  algorithm='ball_tree' #  {‘mutual_proximity’, ‘local_scaling’, ‘dis_sim_local’, None},
                    else:    
                        nbrs = NearestNeighbors(n_neighbors=n_neighbors,hubness = mode, metric = metric ).fit(X2) #  algorithm='ball_tree' #  {‘mutual_proximity’, ‘local_scaling’, ‘dis_sim_local’, None}, 
                    
                    if verbose >= 10:
                        print(str_info,  np.round(time.time()-t0,2),'secs passed')

                    distances, indices = nbrs.kneighbors(X2)
                    edges = np.zeros( (0,2), dtype= int ) 
                    for i in range(1,5):
                        ed = indices[:,[0,i]]
                        edges = np.concatenate( (edges,ed), axis = 0 )

                    g = igraph.Graph( directed = True )
                    g.add_vertices(range(len(X)))
                    g.add_edges(edges )

                    g.to_undirected(mode = 'collapse')
                    louvain_partition = g.community_multilevel()# weights=graph.es['weight'], return_levels=False)
                    modularity1 = g.modularity( louvain_partition )#, we    

                    if verbose >= 10:
                        print("The modularity for Louvain partition is {}".format(modularity1))
                        print(np.round(time.time()-t0,2),'secs passed')

                    list_results.append( (str_info, g) )

                    df_stat.loc[ix4df_stat,'Info'] = str_info
                    df_stat.loc[ix4df_stat,'Louvain modularity'] = modularity1
                    df_stat.loc[ix4df_stat,'Mode'] = mode
                    df_stat.loc[ix4df_stat,'Metric'] =  metric
                    if transform_type == '':
                        df_stat.loc[ix4df_stat,'Transform'] =  'None'
                    else:
                        df_stat.loc[ix4df_stat,'Transform'] = transform_type
                    if dim_reduction is not None:
                        df_stat.loc[ix4df_stat,'PCA'] = str(dim_reduction )
                    else:
                        df_stat.loc[ix4df_stat,'PCA'] = 'None' 

                    ix4df_stat += 1

                    
print(np.round(time.time()-t00,1), np.round( (time.time()-t00)/60,1),  np.round( (time.time()-t00)/3600,1), 'seconds minutes hours total passed')
    
df_stat

In [ ]:
df_stat.to_csv('df_stat')

In [ ]:
df_stat.groupby([ 'Mode',]).mean()

In [ ]:
df_stat.groupby(['PCA', 'Mode']).mean()

In [ ]:
plt.style.use('ggplot')
f1 = 'PCA'
f2 = 'Mode'
for val1 in df_stat[f1].unique():
    fig=plt.figure(figsize=(10,4))
    l = []
    for val2 in ['mutual_proximity', 'Standard KNN graph'   ]: # df_stat[f2].unique():
        mask = (df_stat[f1] == val1) & ( df_stat[f2] == val2)
        v = df_stat['Louvain modularity'][mask]
        l.append(v)
    d = pd.DataFrame(data = (l[0].values,l[1].values )) 
    d = d.transpose()
    d = d.sort_values(0)
    plt.plot(d[0].values, label = 'mutual proximity' )
    plt.plot(d[1].values, label = 'Standard KNN graph' )
    plt.title(f1 + ' ' +str(val1)  )
    plt.ylabel('Louvain modularity' )
    plt.ylabel('dataset' )
    plt.show()
    
    from scipy.stats import wilcoxon
    w, p = wilcoxon(l[0].values , l[1].values )
    #w, p
    print(f1, val1, 'wilcoxon p-value', p)
    
        
    

In [ ]:
df_stat.sort_values( 'Louvain modularity', ascending = False ).head(10)

In [ ]:
df_stat.groupby(['Transform']).mean()

In [ ]:
df_stat.groupby(['Transform', 'Mode',]).mean()

In [ ]:
df_stat.groupby([ 'Transform','Metric',  'Mode',]).mean()

In [ ]:
df_stat.groupby([ 'Transform', 'PCA', 'Metric',  'Mode',]).mean()

In [ ]:
df_stat.groupby([  'Transform','Metric',  'Mode',]).mean()

In [ ]:
df_stat.groupby([  'Transform','PCA',  'Mode',]).mean()

In [ ]:
df_stat.groupby([  'PCA', 'Transform',  'Mode',]).mean()

In [ ]:
df_stat.groupby(['Metric']).mean()

In [ ]:
df_stat.groupby(['Metric', 'Mode',]).mean()

In [ ]:
df_stat.groupby([ 'PCA','Metric',  'Mode',]).mean()

In [ ]:
df_stat.groupby([ 'PCA', 'Mode',]).mean()

In [ ]:
df_stat.groupby([ 'Mode',]).mean()

In [ ]:
df_stat.to_csv('df_stat.csv')

In [ ]:
print(np.round(time.time() - _t00,1),np.round((time.time() - _t00)/60,1),np.round((time.time() - _t00)/3600,1),'total seconds,minutes/hours passed' ) 